In [3]:

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
import json
import requests
import time
import base64
import warnings
import os
import shutil
warnings.filterwarnings('ignore')
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
import re


In [7]:

imgpdf_path=r"C:\Users\rafey\OneDrive - Veracitiz Solutions Pvt Ltd\Desktop\proof-x\Samples of electronic invoices\Samples of electronic invoices\Dataset with valid information\invoice_1.pdf"
def ocr_data_extraction(imgpdf_path):

    if imgpdf_path.split(".")[-1]=='pdf':
        single_img_doc = DocumentFile.from_pdf(imgpdf_path)
    elif imgpdf_path.split(".")[-1]=="png" or imgpdf_path.split(".")[-1]=="JPG":
        single_img_doc = DocumentFile.from_images(imgpdf_path)
        
    model = ocr_predictor(pretrained=True)
    doc = single_img_doc
    result1 = model(doc)
    invoice_data=" "
    for page in result1.pages:
        for block in page.blocks:
            for line in block.lines:
                line1=""
                for word in line.words:
                    text = word.value
                    coordinates = word.geometry  
                    x_min=coordinates[0][0]
                    x_max=coordinates[1][0]
                    y_min=coordinates[0][1]
                    y_max=coordinates[1][1]
                    line1+=text+" "
                invoice_data+=line1+" \n"
 
    result = re.sub(r"(?<=\d),(?=\d)", "", invoice_data)
    return  result

invoice_data=ocr_data_extraction(imgpdf_path)


In [8]:
def load_model():
    with open(r"creds.json", 'r') as f:
        watsonx_cred = json.load(f)

    model = Model(
        model_id=watsonx_cred['model_id'],
        params=watsonx_cred['params'],
        credentials=watsonx_cred['credentials'],
        project_id=watsonx_cred['project_id'],
        space_id=None)
    
    return model
model=load_model()

c:\Users\rafey\OneDrive - Veracitiz Solutions Pvt Ltd\Desktop\proof-x\.venv\Lib\site-packages\ibm_watsonx_ai\foundation_models\model.py:101: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(


In [11]:
def prompt_template(invoice_data):
    prompt = """Task: Categorize and Organize Invoices
    You are provided with extracted invoice data, which contains details such as the bill description, amount, and other relevant information. Your goal is to process this data. 

    Assign each invoice to one of the following predefined categories based on the bill description:

    Healthcare
    Shopping
    Utilities
    Food
    Travel
    Miscellaneous (use this if no specific category fits the bill).
    Generate a JSON File
    Extract and store key details from the invoice, including:

    Date: The date of the invoice.
    Category: The assigned category of the bill.
    Summary: A brief summary of the bill description.
    Bill Amount: The total amount of the bill.

    data=** invoices_data **

    Input:
    The extracted invoice data.

    Output (Example):

    if the data is related to PO
    {
      "pos": [
        { 
        "category": "which category the bill belongs",
          "PO_Number": "Unique identifier for the PO",
          "Order_Date": "When the PO was issued",
          "Delivery_Date": "Expected delivery date",
          "Item_Description": "List of items/services requested",
          "Item_Quantity": "Number of units for each item",
          "Unit_Price": "Price per unit",
          "Subtotal": "Total before taxes and additional charges",
          "Shipping_Address": "Where the goods should be delivered",
          "Billing_Address": "Where the invoice should be sent",
          "Approval_Status": "Approved/Pending/Rejected"
        }
      ]
    }



    if the data is related to invoices

    {
      "invoices": [
        {
          ""
          "category": "which category the bill belongs"
          "Invoice_Number": "Unique identifier for the invoice",
          "Invoice_Date": "When the invoice was issued",
          "Due_Date": "When payment is due",
          "PO_Reference_Number": "If the invoice is linked to a PO",
          "Tax_Details": "e.g., VAT, GST, or other applicable taxes",
          "Line_Item_Details": [
            {
              "Item_Name": "Name of the item",
              "Quantity": "Number of units",
              "Unit_Price": "Price per unit",
              "Line_Total": "Total for this line item"
            }
          ],
          "Discounts": "If any",
          "Payment_Received": "Yes/No",
          "total amount": "total amount paid",
          "total tax" :"total tax paid",
          "summary" : "summary of entire invocie"
        }
      ]
    }

    <endofcode>
    Note: Only return the JSON output in your response"""
    prompt=prompt.replace("invoices_data",invoice_data)
    return prompt

prompt=prompt_template(invoice_data)
    


In [23]:
def llm_output(prompt,model):
    llm_output1=model.generate(prompt)
    generated_response=llm_output1['results'][0]['generated_text'].replace("<endofcode>","")
    return generated_response

generated_response=llm_output(prompt,model)

In [22]:
def data_to_json(generated_response):
    data_string =generated_response
    try:
        start = data_string.index('{')
        end = data_string.rindex('}') + 1
        json_string = data_string[start:end]
        json_data = json.loads(json_string)
        print(list(json_data.keys()))

        if list(json_data.keys())[0]=="invoices":
            with open('output.json', 'r') as file:
                data = json.load(file)
                data['invoices'].append(json_data['invoices'][0])

            with open("output.json","w") as json_file:
                json.dump(data, json_file, indent=4)
        print("JSON data saved successfully:", data)
    except Exception as e:
        print("No valid JSON f ound in the string.",e)
data_to_json(generated_response)


['invoices']
JSON data saved successfully: {'invoices': [{'category': 'Shopping', 'Invoice_Number': '61356291', 'Invoice_Date': '09/06/2012', 'Due_Date': '', 'PO_Reference_Number': '', 'Tax_Details': '10%', 'Line_Item_Details': [{'Item_Name': 'Wine Glasses Goblets Pair Clear', 'Quantity': '500', 'Unit_Price': '1200', 'Line_Total': '6600'}, {'Item_Name': 'With Hooks Stemware Storage', 'Quantity': '400', 'Unit_Price': '2808', 'Line_Total': '12355'}, {'Item_Name': 'Replacement Corkscrew Parts', 'Quantity': '100', 'Unit_Price': '750', 'Line_Total': '825'}, {'Item_Name': 'HOME ESSENTIALS GRADIENT', 'Quantity': '100', 'Unit_Price': '1299', 'Line_Total': '1429'}], 'Discounts': '', 'Payment_Received': '', 'total amount': '21209', 'total tax': '1928', 'summary': 'Purchase of wine glasses, stemware storage, replacement corkscrew parts, and home essentials gradient'}, {'category': 'Shopping', 'Invoice_Number': '61356291', 'Invoice_Date': '09/06/2012', 'Due_Date': '', 'PO_Reference_Number': '', 'T

In [ ]:

def move_file(src_path, dest_folder):
    try:
        os.makedirs(dest_folder, exist_ok=True)
        dest_path = os.path.join(dest_folder, os.path.basename(src_path))
        shutil.move(src_path, dest_path)
        
        print(f"File moved successfully to {dest_path}")
    except FileNotFoundError:
        print(f"Source file not found: {src_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
def rotate_image(image_path, output_path, angle):
    try:
 
        with Image.open(image_path) as img:
            rotated_img = img.rotate(angle, expand=True)
            rotated_img.save(output_path)
            print(f"Image rotated and saved to {output_path}")
    except FileNotFoundError:
        print(f"File not found: {image_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


input_image = "d2eb0f88-2221-476f-a36a-d0123a69189d.jpg"  
output_image = "rotated_example.jpg"  
rotation_angle = 270  
rotate_image(input_image, output_image, rotation_angle)
